# Chapter 3 - Byte Pair Encoding

In this notebook, I analyze the output of my Byte Pair Encoding (BPE) tokenizer, which I trained on a subset of the Wikipedia dataset. The goal was to build a tokenizer with a vocabulary size of 10,000 tokens and then test its effectiveness on a domain-specific dataset—specifically, a set of machine learning research papers.

In [2]:
!pip install datasets transformers huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.5/232.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0


In [3]:
import requests
from datasets import load_dataset
import collections


In [5]:
# Download the Wikipedia dataset from Hugging Face
print("Downloading Wikipedia dataset...")
wikipedia_dataset = load_dataset("olm/olm-wikipedia-20221220")
# wikipedia_dataset = load_dataset("olm/wikipedia")
print("Wikipedia dataset downloaded successfully.")

# Download the Machine Learning Papers dataset from Hugging Face
print("Downloading Machine Learning Papers dataset...")
ml_papers_dataset = load_dataset("CShorten/ML-ArXiv-Papers")
print("Machine Learning Papers dataset downloaded successfully.")

README.md:   0%|          | 0.00/500 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

(…)-00000-of-00042-fb075d4d7bcac9ca.parquet:   0%|          | 0.00/283M [00:00<?, ?B/s]

(…)-00001-of-00042-7aefbfce07009caf.parquet:   0%|          | 0.00/284M [00:00<?, ?B/s]

(…)-00002-of-00042-a930e3e6733eeedd.parquet:   0%|          | 0.00/277M [00:00<?, ?B/s]

...

Wikipedia dataset downloaded successfully.


README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

ML-Arxiv-Papers.csv:   0%|          | 0.00/147M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117592 [00:00<?, ? examples/s]

Machine Learning Papers dataset downloaded successfully.


In [10]:
wikipedia_dataset['train']['text'][0]

ml_papers_text = " ".join(ml_papers_dataset["train"]["abstract"][:50])  # Use a subset for demonstration, did this on colab as it was exploding my local
wikipedia_text = " ".join(wikipedia_dataset["train"]["text"][:70])  # Use a smaller subset for demonstration, did this on colab as it was exploding my local

In [11]:
# ml_papers_text

'  The problem of statistical learning is to construct a predictor of a random\nvariable $Y$ as a function of a related random variable $X$ on the basis of an\ni.i.d. training sample from the joint distribution of $(X,Y)$. Allowable\npredictors are drawn from some specified class, and the goal is to approach\nasymptotically the performance (expected loss) of the best predictor in the\nclass. We consider the setting in which one has perfect observation of the\n$X$-part of the sample, while the $Y$-part has to be communicated at some\nfinite bit rate. The encoding of the $Y$-values is allowed to depend on the\n$X$-values. Under suitable regularity conditions on the admissible predictors,\nthe underlying family of probability distributions and the loss function, we\ngive an information-theoretic characterization of achievable predictor\nperformance in terms of conditional distortion-rate functions. The ideas are\nillustrated on the example of nonparametric regression in Gaussian noise.\n 

In [12]:
# Implement the BPE algorithm (Training Stage)
def bpe_tokenize(text, vocab_size):
  """
  tokenize the text using BPE and return the vocabulary
  @param text: the text to tokenize
  @param vocab_size: the size of the vocabulary
  """
  print("Starting BPE training...")
  # Tokenize the text into words
  words = text.split()
  vocab = collections.defaultdict(int)

  # Initialize the vocabulary with individual characters
  for word in words:
      chars = list(word) + ['</w>']
      for char in chars:
          vocab[tuple(chars)] += 1

  # Perform BPE merges until the desired vocabulary size is reached
  while len(vocab) < vocab_size:
      # Find the most frequent pair of characters
      pairs = collections.defaultdict(int)
      for word, freq in vocab.items():
          for i in range(len(word) - 1):
              pairs[word[i], word[i + 1]] += freq

      if not pairs:
          break

      # Merge the most frequent pair
      best_pair = max(pairs, key=pairs.get)
      new_vocab = collections.defaultdict(int)
      for word, freq in vocab.items():
          new_word = []
          i = 0
          while i < len(word):
              if i < len(word) - 1 and word[i] == best_pair[0] and word[i + 1] == best_pair[1]:
                  new_word.append(word[i] + word[i + 1])
                  i += 2
              else:
                  new_word.append(word[i])
                  i += 1
          new_vocab[tuple(new_word)] += freq

      vocab = new_vocab

  # Return the vocabulary as a list of tokens
  return [''.join(token) for token in vocab.keys()]


In [13]:
# Train BPE on Wikipedia dataset
print("Training BPE on Wikipedia dataset...")
vocab_size = 10000
bpe_vocab = bpe_tokenize(wikipedia_text, vocab_size)
print(f"BPE vocabulary of size {vocab_size} generated.")


Training BPE on Wikipedia dataset...
Starting BPE training...
BPE vocabulary of size 10000 generated.


In [21]:
bpe_vocab[0:50]

['Sayf</w>',
 'al-Din</w>',
 'Ghazi</w>',
 '(II)</w>',
 'ibn</w>',
 'Mawdud</w>',
 '(;</w>',
 'full</w>',
 'name:</w>',
 'II</w>',
 'Zengi;</w>',
 'died</w>',
 '1180)</w>',
 'was</w>',
 'a</w>',
 'Zangid</w>',
 'Emir</w>',
 'of</w>',
 'Mosul,</w>',
 'the</w>',
 'nephew</w>',
 'Nur</w>',
 'ad-Din</w>',
 'Zengi.</w>',
 'He</w>',
 'became</w>',
 'Mosul</w>',
 'in</w>',
 '1170</w>',
 'after</w>',
 'death</w>',
 'his</w>',
 'father</w>',
 'Qutb</w>',
 'Mawdud.</w>',
 'Saif</w>',
 'had</w>',
 'been</w>',
 'chosen</w>',
 'as</w>',
 'successor</w>',
 'under</w>',
 'advice</w>',
 'eunuch</w>',
 '’Abd</w>',
 'al-Masish,</w>',
 'who</w>',
 'wanted</w>',
 'to</w>',
 'keep</w>']

### Examining the BPE Vocabulary

The vocabulary generated by my BPE tokenizer (bpe_vocabulary.txt) contains a mixture of:

- Common words (e.g., "the", "of", "is", "and").

- Named entities, including historical figures and locations.

- Domain-specific terms that emerged from the dataset used for training.

However, due to the training corpus being a subset of Wikipedia, the vocabulary also contains a significant number of historical names and places, which may not be ideal for processing technical texts like machine learning research papers.



In [16]:

# Save the BPE vocabulary to a file
with open("bpe_vocabulary.txt", "w", encoding="utf-8") as vocab_file:
    for token in bpe_vocab:
        vocab_file.write(f"{token}\n")
print("BPE vocabulary saved to 'bpe_vocabulary.txt'.")


BPE vocabulary saved to 'bpe_vocabulary.txt'.


In [17]:
# Inference: Tokenize Machine Learning Papers dataset
print("Tokenizing Machine Learning Papers dataset...")
ml_papers_tokens = bpe_tokenize(ml_papers_text, vocab_size)
print("Tokenization of Machine Learning Papers dataset completed.")


Tokenizing Machine Learning Papers dataset...
Starting BPE training...
Tokenization of Machine Learning Papers dataset completed.


In [22]:
ml_papers_tokens[0:100]

['The</w>',
 'problem</w>',
 'of</w>',
 'statistical</w>',
 'learning</w>',
 'is</w>',
 'to</w>',
 'construct</w>',
 'a</w>',
 'predictor</w>',
 'random</w>',
 'variable</w>',
 '$Y$</w>',
 'as</w>',
 'function</w>',
 'related</w>',
 '$X$</w>',
 'on</w>',
 'the</w>',
 'basis</w>',
 'an</w>',
 'i.i.d.</w>',
 'training</w>',
 'sample</w>',
 'from</w>',
 'joint</w>',
 'distribution</w>',
 '$(X,Y)$.</w>',
 'Allowable</w>',
 'predictors</w>',
 'are</w>',
 'drawn</w>',
 'some</w>',
 'specified</w>',
 'class,</w>',
 'and</w>',
 'goal</w>',
 'approach</w>',
 'asymptotically</w>',
 'performance</w>',
 '(expected</w>',
 'loss)</w>',
 'best</w>',
 'in</w>',
 'class.</w>',
 'We</w>',
 'consider</w>',
 'setting</w>',
 'which</w>',
 'one</w>',
 'has</w>',
 'perfect</w>',
 'observation</w>',
 '$X$-part</w>',
 'sample,</w>',
 'while</w>',
 '$Y$-part</w>',
 'be</w>',
 'communicated</w>',
 'at</w>',
 'finite</w>',
 'bit</w>',
 'rate.</w>',
 'encoding</w>',
 '$Y$-values</w>',
 'allowed</w>',
 'depend</w>'

### Tokenizing Machine Learning Papers

After training my tokenizer, I applied it to a dataset of machine learning papers and examined the output (ml_papers_tokens.txt). Some observations:

**Technical Terms and Subword Splitting:**

- Many key machine learning terms were preserved as full tokens (e.g., "Gaussian", "optimization").

- However, some multi-word terms like "support vector machines" were split into individual words instead of being treated as a single unit.

- Some mathematical symbols and notations were tokenized in a fragmented way, which could impact readability and downstream model performance.

**Comparison to Large Language Models (LLMs):**

- General-purpose LLMs like GPT-4 typically have vocabularies optimized for diverse domains, leading to better preservation of technical terms.

- My tokenizer, being trained on Wikipedia, lacks exposure to specialized scientific texts, which affects how well it represents machine learning terminology.

- LLMs use more sophisticated tokenization strategies, ensuring commonly used technical terms get their own dedicated tokens instead of being broken up.

In [20]:
# Save the tokenized output to a file
with open("ml_papers_tokens.txt", "w", encoding="utf-8") as tokens_file:
    for token in ml_papers_tokens:
        tokens_file.write(f"{token}\n")
print("Tokenized Machine Learning Papers saved to 'ml_papers_tokens.txt'.")


Tokenized Machine Learning Papers saved to 'ml_papers_tokens.txt'.


### Effectiveness for Domain-Specific Texts

From my results, I can infer that general-purpose tokenizers are not always ideal for domain-specific tasks. My BPE tokenizer, trained on Wikipedia, does a decent job with general text but struggles with technical jargon. This suggests that a tokenizer trained on machine learning papers specifically would perform better in preserving domain-specific concepts.

Other things to try:
- Train the BPE tokenizer on a machine learning-specific corpus to see if it better captures technical vocabulary.

- Compare tokenization statistics (e.g., frequency of split vs. full tokens) between my tokenizer and existing LLMs.

- Test the tokenizer on real-world ML tasks to evaluate its impact on downstream NLP performance.

In [ ]:
## Sample Usage (if you want to add your own texts)
your_vocab_size = 10000
bpe_vocab = bpe_tokenize(your_text, your_vocab_size)

